# Base Queries to the ARCGIS Server
Below you will find helper functions to work with the ArcGIS feature server, as well as sample uses of these function that hit the City of Boston feature server (`https://services.arcgis.com/sFnw0xNflSi8J0uh/ArcGIS/rest/services`).

To run the code, first install the `arcgis` module, then run the helper functions cell.

## Helper Functions
Run this first

In [0]:
!pip install arcgis==1.4.0

In [0]:
from arcgis.gis import *
from arcgis.features import FeatureLayer
from arcgis.geocoding import geocode
from arcgis import geometry
from arcgis import network
from pprint import pprint
import datetime
import requests

gis = GIS()
QUERY = { 'where': '1=1', 'out_sr': '4326' }
DAY = datetime.datetime.today().strftime('%A')
MILE = 1600
RESOURCEID = "12cb3883-56f5-47de-afa5-3b1cf61b257b"
CRIME_INCIDENTS_SQL_URL = "https://data.boston.gov/api/3/action/datastore_search_sql"
QUERY_LIMIT = 5


def get_crime_incident_response(address):
    """
    Executes and returns the crime incident request response

    :param address: address to query
    :return: the raw json response

    """
    url_parameters = {"sql": _build_query_string(address)}
    with requests.Session() as session:
        response = session.get(CRIME_INCIDENTS_SQL_URL, params=url_parameters)

    if response.status_code == requests.codes.ok:
        return response.json()
    return {}

  
def _build_query_string(address):
    """
    Builds the SQL query given an address

    :param address: address to query
    :return: a SQL query string

    """
    coordinates = _get_coordinates_for_address(address)
    return """SELECT * FROM "{}" WHERE "lat" LIKE '{}%' AND \
        "long" LIKE '{}%' LIMIT {}""" \
        .format(RESOURCEID,
                coordinates[0],
                coordinates[1],
                QUERY_LIMIT)
  

def get_features_from_feature_server(url, query):
  """
  Given a url to a City of Boston Feature Server, return a list
  of Features (for example, parking lots that are not full)

  :param url: url for Feature Server
  :param query: query to select features (example: "Spaces > 0")
  :return: list of all features returned from the query
  """
  features = []
  f = FeatureLayer(url = url)
  feature_set = f.query(**query)
  for feature in feature_set:
      features.append(feature.as_dict)
  return features


def geocode_address(m_address):
    """
    :param m_address: address of interest in street form
    :return: address in coordinate (X and Y) form
    """
    m_address = m_address + ", City: Boston, State: MA"
    m_location = geocode(address=m_address)[0]
    return m_location['location']
  
  
def get_geodesic_distance(feature1, feature2):
  geometry1 = feature1#['geometry']
  geometry2 = feature2['geometry']
  spation_ref = {"wkid" : 4326}
  return geometry.distance(spation_ref, 
                           geometry1, 
                           geometry2,
                           distance_unit='', 
                           geodesic=True)['distance']


def get_feature_location(url, query):
  return get_features_from_feature_server(url, query)
  

def _get_coordinates_for_address(address):
    """
    Populates the GPS coordinates for the provided address

    :param address: address to query
    :return: a tuple of the form (lat, long)

    """
    coordinates = geocode_address(address)
    _lat = "{:.2f}".format(float(coordinates['y']))
    _long = "{:.2f}".format(float(coordinates['x']))
    return _lat, _long
  
  



## Grocery Stores in Boston

In [0]:
grocery_url = "https://services.arcgis.com/sFnw0xNflSi8J0uh/ArcGIS/rest/services/Supermarkets_GroceryStores/FeatureServer/0"

# this will return a list of all grocery stores Boston
grocery = get_feature_location(grocery_url, QUERY)
pprint(grocery[0])
print()

for g in grocery:
  print(g['attributes']['Store'] + ' is located in ' 
        + g['attributes']['Neighborho'] + ' at ' 
        + g['attributes']['Address'])


{'attributes': {'Address': '370 Western Avenue',
                'FID': 1,
                'Lat': 42.3609803747,
                'Lon': -71.137830016,
                'Neighborho': 'Allston',
                'ObjectID': 1,
                'Store': 'Star Market',
                'Type': 'Supermarket',
                'Zip': '02135'},
 'geometry': {'x': -71.13783063274357, 'y': 42.360989045712415}}

Star Market is located in Allston at 370 Western Avenue
Hong Kong Supermarket is located in Allston at 1 Brighton Avenue
Star Market is located in Back Bay at 53 Huntington Avenue
Trader Joe's is located in Back Bay at 899 Boylston Street
Whole Foods Market is located in Brighton at 15 Washington Street
Star Market is located in Brighton at 1065 Commonwealth Avenue
Super Stop & Shop is located in Allston at 60 Everett Street
America's Food Basket is located in Dorchester at 500 Geneva Avenue
America's Food Basket is located in Dorchester at 576 Washington Street
Star Market is located in Dorc

## Food trucks today

In [0]:
food_trucks_url = 'https://services.arcgis.com/sFnw0xNflSi8J0uh/arcgis/rest/services/food_trucks_schedule/FeatureServer/0'
address = geocode_address('120 Boylston St')

# this will return a list of all available food trucks in Boston
trucks = get_feature_location(food_trucks_url, QUERY)
trucks_today = []
for t in trucks:
  if(t['attributes']['Day'] == DAY and t['attributes']['Time'] == 'Lunch'):
    trucks_today.append(t)

pprint(trucks_today[0])    

for t in trucks_today:
  distance = get_geodesic_distance(address, t)
  if distance <= MILE:
    print(t['attributes']['Truck'] + ' is located at ' + \
          t['attributes']['Loc'] + ' between ' + \
          t['attributes']['Start_time'] + ' and ' + \
          t['attributes']['End_time'] + '\n')


{'attributes': {'CreationDate': 1520268574231,
                'Creator': '143525_boston',
                'Day': 'Wednesday',
                'EditDate': 1520268574231,
                'Editor': '143525_boston',
                'End_time': '3:00:00 PM',
                'FID': 10,
                'GlobalID': 'd4a84cb9-8685-461a-ab5e-a19260656933',
                'Hours': '11 a.m. - 3 p.m.',
                'Link': 'http://www.morockinfusion.com/',
                'Loc': 'Chinatown Park',
                'Management': 'Rose Kennedy Greenway Conservancy',
                'Notes': ' ',
                'POINT_X': -7910324.35486,
                'POINT_Y': 5213745.7545,
                'Site_num': 0,
                'Start_time': '11:00:00 AM',
                'Time': 'Lunch',
                'Title': 'Greenway',
                'Truck': "Mo'Rockin Fusion"},
 'geometry': {'x': -71.05965270349564, 'y': 42.351281064296}}
Mo'Rockin Fusion is located at Chinatown Park between 11:00:00 AM and 3

## Moving Permit

In [0]:
moving_trucks_url = 'https://services.arcgis.com/sFnw0xNflSi8J0uh/ArcGIS/rest/services/Moving_Truck_Permits/FeatureServer/0'

# get moving permits
permits = get_feature_location(moving_trucks_url, QUERY)

active_permits = []
for p in permits:
  if p['attributes']['Status'] != 'EXPIRED':
    active_permits.append(p)

print('Hang on, this might take a while...')
pprint(active_permits[0])

Hang on, this might take a while...
{'attributes': {'Applican_1': 2115,
                'Applicant1': 'MA',
                'Applicant_': 'Boston',
                'Applicatio': 'InPersonPermit',
                'City': 'Boston',
                'Comments': 'STAND MOVING VEHICLES AT CURB; 23 SPACES;FOR '
                            'STUDENT MOVE-IN // 7AM TO 9PM;//;; RENEWAL',
                'Descriptio': 'Moving Trucks',
                'Expiration': 1504310400000,
                'Is_contrac': 'N',
                'Issued_dat': None,
                'Lat': 42.342421,
                'Long': -71.103967,
                'OBJECTID': 5355,
                'OccupancyT': ' ',
                'PermitNumb': 'OCCU-170061',
                'PermitType': 'Street Occupancy Permit',
                'State': 'MA',
                'Status': 'OPEN',
                'Total_Fees': 1547,
                'Worktype': 'Movetrucks',
                'Zip': 2215},
 'geometry': {'x': -71.10396700027678, 'y':

## Farmers Markets

In [0]:
farmers_market_url = "https://services.arcgis.com/sFnw0xNflSi8J0uh/ArcGIS/rest/services/Farmers_Markets_and_Fresh_Trucks/FeatureServer/0"

# this will return a list of all farmers markets in Boston
markets = get_feature_location(farmers_market_url, QUERY)

markets_unique = []
for m in markets:
  #print(m)
  if m not in markets_unique and m['attributes']['Day_'] == DAY:
    markets_unique.append(m)
    #print(markets_unique)


print('Avaiable farmers markets today are:')
for m in markets_unique:
  print(m['attributes']['Name'] + ' located at ' + 
        m['attributes']['Location'] + ' from ' + 
        m['attributes']['Start'])

Avaiable farmers markets today are:
Boston Public Market located at Haymarket T Station from 8:00am - 8:00pm
Social Saturdays at the Codman Square Farmers Market located at Codman Square Park/Common from 10:00am - 1:00pm
Egleston Farmers Market located at Across from Sam Adams Brewery from 10:00am - 2:00pm
Fields Corner Farmers Market located at Fields Corner Shopping Center from 9:00am - 12:00pm
Mattapan Square Farmers Market located at Church of the Holy Spirit from 11:00am - 1:00pm
Roslindale Village Main Street located at Near the Roslindale Commuter rail stop:Needham Line from 9:00am - 1:30pm
Jamaica Plain Farmers Market located at Bank Of America parking lot from 12:00pm - 3:00pm
Maverick Square T Stop located at   from 11:30 am - 3:30 pm
Orient Heights located at   from 4:00 pm - 5:00 pm
